In [ ]:
cd("/home/gridsan/aramadhan/tmp")
pwd()

In [1]:
using Printf

In [2]:
# Source: https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/src/trials.jl
function prettytime(t)
    if t < 1e3
        value, units = t, "ns"
    elseif t < 1e6
        value, units = t / 1e3, "μs"
    elseif t < 1e9
        value, units = t / 1e6, "ms"
    else
        s = t / 1e9
        if s < 60
            value, units = s, "s"
        else
            value, units = (s / 60), "min"
        end
    end
    return string(@sprintf("%.3f", value), " ", units)
end

prettytime (generic function with 1 method)

In [3]:
# https://github.com/IainNZ/Humanize.jl/blob/master/src/Humanize.jl

import Printf: @sprintf

const SUFFIXES = Dict(
    :dec => [" B", " kB", " MB", " GB", " TB", " PB", " EB", " ZB", " YB"],
    :bin => [" B", " KiB", " MiB", " GiB", " TiB", " PiB", " EiB", " ZiB", " YiB"],
    :gnu => ["", "K", "M", "G", "T", "P", "E", "Z", "Y"]
)
const BASES = Dict(:dec => 1000, :bin => 1024, :gnu => 1024)



"""
    datasize(value::Number; style=:dec, format="%.1f")
Format a number of bytes in a human-friendly format (eg. 10 kB).
    style=:dec - default, decimal suffixes (kB, MB), base 10^3
    style=:bin - binary suffixes (KiB, MiB), base 2^10
    style=:gnu - GNU-style (ls -sh style, K, M), base 2^10
"""
function datasize(value::Number; style=:dec, format="%.1f")::String
    suffix = SUFFIXES[style]
    base = float(BASES[style])
    bytes = float(value)
    unit = base
    biggest_suffix = suffix[1]
    for power in 1:length(suffix)
        unit = base^power
        biggest_suffix = suffix[power]
        bytes < unit && break
    end
    value = base * bytes / unit
    format = "$format%s"
    return @eval @sprintf($format, $value, $biggest_suffix)
end

datasize

In [ ]:
N = 256
A = rand(N, N, N)

for cl in 0:9
    tic = time_ns()
    
    filename = "compress" * string(cl) * ".nc"
    varname  = "rands"
    attribs  = Dict("units"   => "m/s")

    nccreate(filename, varname,
             "x1", collect(1:N), Dict("units"=>"m"),
             "x2", collect(1:N), Dict("units"=>"m"),
             "x3", collect(1:N), Dict("units"=>"m"),
             atts=attribs, chunksize=(1,16,256), compress=cl)
    ncwrite(A, filename, varname)
    ncclose(filename)
    
    toc = time_ns()
    
    ts = prettytime(toc - tic)
    fs = datasize(filesize(filename); style=:bin, format="%.3f")
    println("Compression level $cl: $ts $fs")
end

In [ ]:
ncclose(filename)

In [4]:
using Pkg
cd("/home/gridsan/aramadhan/tmp/NetCDF.jl")
Pkg.activate(".");
pwd()

"/home/gridsan/aramadhan/tmp/NetCDF.jl"

In [9]:
using NetCDF

┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/NetCDF/WUyLg.ji for NetCDF [30363a11-5582-574a-97bb-aa9a979735b9]
└ @ Base loading.jl:1184
┌ Warning: Package NetCDF does not have Libdl in its dependencies:
│ - If you have NetCDF checked out for development and have
│   added Libdl as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with NetCDF
└ Loading Libdl into NetCDF from project dependency, future warnings for NetCDF are suppressed.


ErrorException: importing NetCDF into Main conflicts with an existing identifier

In [15]:
N = 256
A = Float64.(rand(1:10, N, N, N))

for cl in 0:9
    tic = time_ns()
    
    filename = "compress" * string(cl) * ".nc"
    varname  = "rands"
    attribs  = Dict("units"   => "m/s")

    nccreate(filename, varname,
             "x1", collect(1:N), Dict("units"=>"m"),
             "x2", collect(1:N), Dict("units"=>"m"),
             "x3", collect(1:N), Dict("units"=>"m"),
             atts=attribs, chunksize=(256,16,1), compress=cl)
    ncwrite(A, filename, varname)
    ncclose(filename)
    
    toc = time_ns()
    
    ts = prettytime(toc - tic)
    fs = datasize(filesize(filename); style=:bin, format="%.3f")
    println("Compression level $cl: $ts $fs")
end

Compression level 0: 866.125 ms 128.280 MiB
Compression level 1: 930.343 ms 12.469 MiB
Compression level 2: 985.205 ms 12.097 MiB
Compression level 3: 1.078 s 11.716 MiB
Compression level 4: 1.407 s 11.421 MiB
Compression level 5: 1.557 s 11.212 MiB
Compression level 6: 1.918 s 11.010 MiB
Compression level 7: 2.348 s 10.952 MiB
Compression level 8: 4.791 s 10.858 MiB
Compression level 9: 6.997 s 10.843 MiB
